In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from portal_analytics.api import AnalyticsApi
import portal_analytics.parser as parser
import portal_analytics.create_rasters as cr
from shapely.geometry import box, Point
import json
import pandas as pd

In [3]:
with open('items.json') as file:
      items = json.load(file)
layers=  items["layers"]
for layer in layers:
    print(layer)

mineral_occurrences
mines
mineral_tenements
scanned_250K_maps
nvcl
all_boreholes


In [4]:
KEY_FILE = 'GeosciencePortal-eeac25f8efb3.json'
ACCOUNT_NAME = 'AUSGIN website'
PROPERTY_NAME = 'AUSGIN'
PROFILE_NAME = 'AUSGIN - exclude internal traffic'


In [5]:
connection=AnalyticsApi(KEY_FILE,ACCOUNT_NAME,PROPERTY_NAME,PROFILE_NAME)

In [17]:
rows, headers = connection.get_results()

In [18]:
results

([['Add:KMLRecord',
   'Layer:4APL-4BLH Bald Hills.kml',
   'Filter:{"bbox":"{\\"westBoundLongitude\\":\\"152\\",\\"southBoundLatitude\\":\\"-27.6\\",\\"eastBoundLongitude\\":\\"154\\",\\"northBoundLatitude\\":\\"-26.8\\",\\"crs\\":\\"EPSG:4326\\"}"}',
   '1'],
  ['Add:KMLRecord',
   'Layer:4APL-4BLH Bald Hills.kmz',
   'Filter:{"bbox":"{\\"westBoundLongitude\\":\\"152\\",\\"southBoundLatitude\\":\\"-27.7\\",\\"eastBoundLongitude\\":\\"154\\",\\"northBoundLatitude\\":\\"-26.8\\",\\"crs\\":\\"EPSG:4326\\"}"}',
   '1'],
  ['Add:KMLRecord',
   'Layer:574TallebudgeraCkRd.kmz',
   'Filter:{"bbox":"{\\"westBoundLongitude\\":\\"153.4\\",\\"southBoundLatitude\\":\\"-28.2\\",\\"eastBoundLongitude\\":\\"153.5\\",\\"northBoundLatitude\\":\\"-28.1\\",\\"crs\\":\\"EPSG:4326\\"}"}',
   '1'],
  ['Add:KMLRecord',
   'Layer:All bores map.kml',
   'Filter:{"bbox":"{\\"westBoundLongitude\\":\\"113\\",\\"southBoundLatitude\\":\\"-25\\",\\"eastBoundLongitude\\":\\"122\\",\\"northBoundLatitude\\":\\"-20\\",

In [20]:
raw_results = parser.create_dataframe(rows, headers)
raw_results.to_excel("raw_results.xlsx")

In [21]:
data_frame = parser.process_dataframe(raw_results)
data_frame["ga:totalEvents"]=data_frame["ga:totalEvents"].apply(int)
add_layers=data_frame[data_frame["ga:eventCategory"]=="Add:KnownLayer"].groupby(by="ga:eventAction")["ga:totalEvents"].sum()

In [22]:
pd.DataFrame(add_layers).sort_values("ga:totalEvents", ascending=False).to_excel("add_layers.xlsx")

In [23]:
query=raw_results[raw_results["ga:eventCategory"]=="Query"]

In [24]:
query["ga:eventAction"].unique();

In [25]:
gdf = parser.create_geodataframe(data_frame)
%matplotlib inline

In [26]:
map_centre=Point(133.3,-26)

australia_bounds = box(110,-45,155,-10)
buff=map_centre.buffer(1)


In [27]:
#gdf["distance"] = gdf["shape"].centroid.distance(map_centre)
gdf["area"] = gdf["shape"].area
gdf["weighting"] = australia_bounds.area/ gdf["shape"].area * gdf["ga:totalEvents"]
gdf = gdf[gdf["shape"].centroid.intersects(australia_bounds)]
#gdf = gdf[gdf["shape"].centroid.intersects(buff)]

gdf = gdf[gdf["area"] <= australia_bounds.area]
#gdf = gdf[gdf["area"] > 0]
#gdf


x=gdf[gdf['ga:eventAction'] == "Layer:Scanned 250K Geological Map Index"]


In [28]:
cr.create_spatial_data(gdf)



Mineral Occurrences
output log2
7.036173612553485
(20, 60)
(115.0, -28.0, 121.0, -26.0)
60 20 0.1
output log2/mineral_occurrences/1277.tif
11.621136113274641
(5, 10)
(116.9, -29.7, 117.9, -29.2)
10 5 0.1
output log2/mineral_occurrences/1278.tif
13.872674880270596
(3, 7)
(117.5, -27.3, 118.2, -27.0)
7 3 0.1
output log2/mineral_occurrences/1279.tif
14.26499230304935
(1, 4)
(117.6, -21.4, 118.0, -21.2)
4 1 0.1
output log2/mineral_occurrences/1280.tif
11.358101707440852
(5, 20)
(117.0, -27.4, 119.0, -26.8)
20 5 0.1
output log2/mineral_occurrences/1281.tif
9.036173612553485
(10, 30)
(117.0, -28.0, 120.0, -27.0)
30 10 0.1
output log2/mineral_occurrences/1282.tif
12.872674880270596
(3, 7)
(118.3, -27.2, 119.0, -26.9)
7 3 0.1
output log2/mineral_occurrences/1283.tif
17.26499230304948
(0, 0)
17.26499230304948
(119.2, -25.4, 119.3, -25.3)
1 1 0.1
output log2/mineral_occurrences/1284.tif
14.680029802328203
(2, 2)
(121.7, -32.5, 122.0, -32.3)
2 2 0.1
output log2/mineral_occurrences/1285.tif
4.9772

(230, 400)
(105.0, -35.0, 145.0, -12.0)
400 230 0.1
output log2/mineral_occurrences/4416.tif
2.360608563051422
(230, 400)
(105.0, -35.0, 145.0, -12.0)
400 230 0.1
output log2/mineral_occurrences/4417.tif
1.775646062330266
(230, 400)
(105.0, -35.0, 145.0, -12.0)
400 230 0.1
output log2/mineral_occurrences/4418.tif
5.813781191217037
(40, 70)
(147.0, -32.0, 154.0, -28.0)
70 40 0.1
output log2/mineral_occurrences/4420.tif
2.7693870718585836
(110, 210)
(113.0, -34.0, 134.0, -23.0)
210 110 0.1
output log2/mineral_occurrences/4421.tif
16.264992303049375
(0, 2)
16.264992303049375
(119.6, -21.9, 119.8, -21.8)
1 1 0.1
output log2/mineral_occurrences/4422.tif
6.036173612553485
(40, 60)
(113.0, -29.0, 119.0, -25.0)
60 40 0.1
output log2/mineral_occurrences/6721.tif
14.264992303049326
(2, 4)
(117.8, -35.1, 118.2, -34.9)
4 2 0.1
output log2/mineral_occurrences/6722.tif
6.714245517666122
(30, 50)
(129.0, -24.0, 134.0, -21.0)
50 30 0.1
output log2/mineral_occurrences/6723.tif
3.141355849245542
(170, 2

(143.9, -36.8, 144.7, -36.5)
7 2 0.1
output log2/mineral_occurrences/9195.tif
15.264992303049427
(0, 4)
15.264992303049427
(144.2, -36.8, 144.6, -36.7)
1 1 0.1
output log2/mineral_occurrences/9196.tif
21.90884849282569
(0, 0)
21.90884849282569
(138.79000000000002, -34.809999999999995, 138.81, -34.79)
1 1 0.1
output log2/mineral_occurrences/9197.tif
9.773139206719682
(9, 20)
(149.0, -35.2, 151.0, -34.3)
20 9 0.1
output log2/mineral_occurrences/9198.tif
12.264992303049375
(3, 7)
(118.2, -25.7, 119.0, -25.3)
7 3 0.1
output log2/mineral_occurrences/9199.tif
3.2117451771369394
(100, 170)
(115.0, -23.0, 132.0, -13.0)
170 100 0.1
output log2/mineral_occurrences/9202.tif
11.943064208162035
(10, 7)
(145.9, -27.0, 146.7, -26.0)
7 10 0.1
output log2/mineral_occurrences/9206.tif
6.22881869049588
(70, 30)
(150.0, -33.0, 153.0, -26.0)
30 70 0.1
output log2/mineral_occurrences/9207.tif
11.135709286104401
(6, 10)
(116.0, -21.9, 117.0, -21.2)
10 6 0.1
output log2/mineral_occurrences/9208.tif
2.33573389

(118.0, -21.0, 121.0, -20.0)
30 10 0.1
output log2/mines/1435.tif
8.621136113274641
(20, 20)
(125.0, -30.0, 127.0, -28.0)
20 20 0.1
output log2/mines/1436.tif
1.5336732720243016
(160, 340)
(128.0, -32.0, 162.0, -16.0)
340 160 0.1
output log2/mines/1437.tif
9.773139206719687
(9, 20)
(129.0, -31.8, 131.0, -30.9)
20 9 0.1
output log2/mines/1438.tif
16.26499230304943
(1, 1)
(130.3, -31.6, 130.5, -31.5)
1 1 0.1
output log2/mines/1439.tif
9.036173612553485
(10, 30)
(131.0, -32.0, 134.0, -31.0)
30 10 0.1
output log2/mines/1440.tif
13.358101707440843
(3, 5)
(132.3, -32.0, 132.8, -31.7)
5 3 0.1
output log2/mines/1441.tif
11.358101707440843
(6, 10)
(133.0, -32.6, 134.0, -32.0)
10 6 0.1
output log2/mines/1442.tif
22.90884849282518
(0, 0)
22.90884849282518
(138.49, -30.53, 138.51, -30.52)
1 1 0.1
output log2/mines/1443.tif
17.26499230304943
(1, 0)
17.26499230304943
(138.5, -30.6, 138.6, -30.5)
1 1 0.1
output log2/mines/1444.tif
11.621136113274641
(5, 10)
(140.0, -27.6, 141.0, -27.1)
10 5 0.1
outpu

(50, 20)
(143.0, -32.0, 145.0, -27.0)
20 50 0.1
output log2/mines/6933.tif
5.621136113274641
(40, 80)
(147.0, -31.0, 155.0, -27.0)
80 40 0.1
output log2/mines/6934.tif
10.943064208162008
(7, 10)
(147.0, -33.8, 148.0, -33.0)
10 7 0.1
output log2/mines/6935.tif
7.621136113274641
(20, 40)
(148.0, -34.0, 152.0, -32.0)
40 20 0.1
output log2/mines/6936.tif
7.2992080183872785
(50, 40)
(134.0, -19.0, 138.0, -14.0)
40 50 0.1
output log2/mines/6938.tif
12.26499230304935
(3, 8)
(138.5, -34.9, 139.3, -34.5)
8 3 0.1
output log2/mines/6939.tif
7.621136113274641
(20, 40)
(115.0, -35.0, 119.0, -33.0)
40 20 0.1
output log2/mines/6940.tif
7.621136113274641
(20, 40)
(115.0, -35.0, 119.0, -33.0)
40 20 0.1
output log2/mines/6942.tif
7.621136113274641
(20, 40)
(115.0, -35.0, 119.0, -33.0)
40 20 0.1
output log2/mines/6943.tif
10.621136113274641
(10, 20)
(117.0, -24.0, 119.0, -23.0)
20 10 0.1
output log2/mines/6944.tif
7.621136113274641
(20, 40)
(115.0, -35.0, 119.0, -33.0)
40 20 0.1
output log2/mines/6945.ti

...

(136.0, -39.0, 161.0, -28.0)
250 110 0.1
output log2/nvcl/25932.tif
4.5178483048626195
(110, 250)
(136.0, -41.0, 161.0, -30.0)
250 110 0.1
output log2/nvcl/25933.tif
22.90884849282569
(0, 0)
22.90884849282569
(145.59, -30.88, 145.60999999999999, -30.87)
1 1 0.1
output log2/nvcl/25934.tif
1.5336732720243016
(160, 340)
(137.0, -40.0, 171.0, -24.0)
340 160 0.1
output log2/nvcl/25935.tif
2.5178483048626195
(110, 250)
(136.0, -41.0, 161.0, -30.0)
250 110 0.1
output log2/nvcl/25937.tif
9.036173612553485
(10, 30)
(141.0, -28.0, 144.0, -27.0)
30 10 0.1
output log2/nvcl/25938.tif
11.943064208161983
(5, 8)
(116.6, -32.4, 117.4, -31.9)
8 5 0.1
output log2/nvcl/29678.tif
16.264992303049326
(1, 2)
(116.8, -32.2, 117.0, -32.1)
2 1 0.1
output log2/nvcl/29679.tif
13.09506730160708
(2, 5)
(118.4, -32.8, 119.0, -32.5)
5 2 0.1
output log2/nvcl/29680.tif
2.0742416533870043
(220, 340)
(119.0, -28.0, 153.0, -6.0)
340 220 0.1
output log2/nvcl/29681.tif
13.943064208162008
(1, 5)
(121.6, -30.8, 122.1, -30.6)
5

(149.4, -32.01, 149.5, -31.99)
1 1 0.1
output log2/all_boreholes/3317.tif
4.491853096329675
(70, 100)
(142.0, -40.0, 152.0, -33.0)
100 70 0.1
output log2/all_boreholes/3318.tif
22.90884849282518
(0, 0)
22.90884849282518
(145.74, -17.16, 145.75, -17.139999999999997)
1 1 0.1
output log2/all_boreholes/3321.tif
16.26499230304943
(1, 1)
(151.8, -26.3, 152.0, -26.2)
1 1 0.1
output log2/all_boreholes/3322.tif
19.586920397936584
(0, 0)
19.586920397936584
(153.0, -27.51, 153.1, -27.49)
1 1 0.1
output log2/all_boreholes/3323.tif
2.7022728760000465
(110, 220)
(107.0, -34.0, 129.0, -23.0)
220 110 0.1
output log2/all_boreholes/3324.tif
16.26499230304943
(1, 1)
(115.9, -32.2, 116.1, -32.1)
1 1 0.1
output log2/all_boreholes/3325.tif
19.586920397937096
(0, 0)
19.586920397937096
(116.0, -32.11, 116.1, -32.09)
1 1 0.1
output log2/all_boreholes/3326.tif
9.621136113274641
(10, 20)
(137.0, -32.0, 139.0, -31.0)
20 10 0.1
output log2/all_boreholes/3328.tif
19.586920397936687
(0, 1)
19.586920397936687
(138.7,

30.552704682631166
(0, 0)
30.552704682631166
(145.02, -28.169, 145.021, -28.168)
1 1 0.1
output log2/all_boreholes/8272.tif
30.552704682631166
(0, 0)
30.552704682631166
(145.02, -28.169, 145.021, -28.168)
1 1 0.1
output log2/all_boreholes/8273.tif
3.3637482705819894
(90, 170)
(137.0, -42.0, 154.0, -33.0)
170 90 0.1
output log2/all_boreholes/8274.tif
7.621136113274641
(20, 40)
(143.0, -39.0, 147.0, -37.0)
40 20 0.1
output log2/all_boreholes/8275.tif
2.5767419939161877
(120, 220)
(136.0, -28.0, 158.0, -16.0)
220 120 0.1
output log2/all_boreholes/8276.tif
7.2992080183872785
(20, 50)
(147.0, -25.0, 152.0, -23.0)
50 20 0.1
output log2/all_boreholes/8277.tif
0.802553935793783
(210, 430)
(127.0, -40.0, 170.0, -19.0)
430 210 0.1
output log2/all_boreholes/10605.tif
2.0588936890535687
(140, 270)
(113.0, -35.0, 140.0, -21.0)
270 140 0.1
output log2/all_boreholes/10608.tif
6.451211111832329
(30, 60)
(115.0, -24.0, 121.0, -21.0)
60 30 0.1
output log2/all_boreholes/10609.tif
21.908848492823125
(0, 0

(134.0, -30.0, 156.0, -20.0)
220 100 0.1
output log2/all_boreholes/17513.tif
5.621136113274641
(40, 80)
(136.0, -22.0, 144.0, -18.0)
80 40 0.1
output log2/all_boreholes/17514.tif
26.23077658771613
(0, 0)
26.23077658771613
(138.662, -34.794, 138.682, -34.793)
1 1 0.1
output log2/all_boreholes/17515.tif
2.3923174227787602
(120, 250)
(138.0, -39.0, 163.0, -27.0)
250 120 0.1
output log2/all_boreholes/17516.tif
10.943064208161996
(8, 10)
(145.0, -43.1, 146.0, -42.3)
10 8 0.1
output log2/all_boreholes/17517.tif
12.287712379549488
(6, 8)
(146.8, -20.0, 147.7, -19.3)
8 6 0.1
output log2/all_boreholes/17518.tif
11.510104800885886
(6, 9)
(146.9, -20.0, 147.8, -19.4)
9 6 0.1
output log2/all_boreholes/17519.tif
21.90884849282518
(0, 0)
21.90884849282518
(153.09, -27.21, 153.10999999999999, -27.189999999999998)
1 1 0.1
output log2/all_boreholes/17520.tif
6.22881869049588
(30, 70)
(141.0, -38.0, 148.0, -35.0)
70 30 0.1
output log2/all_boreholes/17527.tif
9.621136113274641
(10, 20)
(150.0, -34.5, 152

(270, 460)
(124.0, -44.0, 170.0, -17.0)
460 270 0.1
output log2/all_boreholes/28534.tif
17.26499230304943
(1, 0)
17.26499230304943
(129.3, -15.3, 129.4, -15.2)
1 1 0.1
output log2/all_boreholes/28535.tif
20.586920397936584
(0, 0)
20.586920397936584
(131.5, -22.12, 131.6, -22.11)
1 1 0.1
output log2/all_boreholes/28536.tif
13.943064208162008
(1, 5)
(133.4, -27.4, 133.9, -27.2)
5 1 0.1
output log2/all_boreholes/28537.tif
16.264992303049222
(1, 2)
(139.6, -34.1, 139.8, -34.0)
2 1 0.1
output log2/all_boreholes/28538.tif
22.90884849282569
(0, 0)
22.90884849282569
(139.69, -34.04, 139.70999999999998, -34.03)
1 1 0.1
output log2/all_boreholes/28539.tif
5.451211111832329
(40, 90)
(140.0, -36.0, 149.0, -32.0)
90 40 0.1
output log2/all_boreholes/28540.tif
7.621136113274641
(20, 40)
(141.0, -11.0, 145.0, -9.0)
40 20 0.1
output log2/all_boreholes/28541.tif
14.095067301607012
(2, 3)
(146.1, -34.4, 146.4, -34.1)
3 2 0.1
output log2/all_boreholes/28542.tif
19.586920397936584
(0, 0)
19.586920397936584

In [15]:
x=1